In [1]:
import os
import sys
import argparse
from collections import OrderedDict
from collections import OrderedDict as OD
import csv
import json

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import hist
import mplhep as hep

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval # hyperparameter optimization
import pickle


from CommonTools import (
    convert_Et_to_logEt, 
    convert_logEt_to_Et, 
    convert_GenEt_to_GenEtByL1Et, 
    convert_GenEtByL1Et_to_GenEt, 
    convert_GenEt_to_logGenEtByL1Et, 
    convert_logGenEtByL1Et_to_GenEt,
    convert_CaloToolMPEta_to_IEta,
    prepareDataframeForSFs,
    GaussianFunction,
    calculate_errorOfRatio
)

JEC_SFs_files_dict = {
    "2022_ChunkyDonut": {
        "HBEF": {
            "PtAll": {
                "fileName" : '/home/siddhesh/Work/CMS/L1_Trigger_Work/L1T_ServiceTask/hcalPUsub/myAna/hcalPUsub_v5_20220311/run_3/l1JetShapesStudies/data/L1T_Jet_SFs_Run3_QCD_Pt15to7000_PFA1p_CMSSW12_6_0_pre1_nVtxAll_20220925_v3_HyperOpt_L1JetEt_PUS_ChunkyDonut.csv'
            }
        }
    },
    
    "2023_ChunkyDonut": {
        "saveAs": '../data/L1T_Jet_SFs_2023_QCDP_126X_mcRun3_2023_13_1_0_pre2_HBZS0p5_20230330_L1JetEt_PUS_ChunkyDonut_v0_HBE_logGenEtByL1Et_atPU33_HF_GenEtByL1Et_atPU33_woHyperParamOptmz.csv',
        "HBE": {
            "PtAll": {
                "fileName" : '../data/BDTModel_vL1JetEt_PUS_ChunkyDonut_GenJetEt_MLTarget_logGenEtByL1Et_dataFrac1.00_20230403_HBEF_PtAll.pkl',
                "MLTarget": 'logGenEtByL1Et',
                "PUForSFComputation": 33
            },
        },
        "HF": {
            "PtAll": {
                "fileName" : '../data/BDTModel_vL1JetEt_PUS_ChunkyDonut_GenJetEt_MLTarget_GenEtByL1Et_dataFrac1.00_20230403_HBEF_PtAll.pkl',
                "MLTarget": 'GenEtByL1Et',
                "PUForSFComputation": 33
            },
        },        
    },
    
    "2022_PhiRing": {
        "HBEF": {
            "PtAll": {
                "fileName" : '/home/siddhesh/Work/CMS/L1_Trigger_Work/L1T_ServiceTask/hcalPUsub/myAna/hcalPUsub_v5_20220311/run_3/l1JetShapesStudies/data/L1T_Jet_SFs_Run3_QCD_Pt15to7000_PFA1p_CMSSW12_6_0_pre1_nVtxAll_20220925_v3_HyperOpt_L1JetEt_PUS_PhiRing.csv'
            }
        }
    },
    
    "2023_PhiRing": {
        "saveAs": '../data/L1T_Jet_SFs_2023_QCDP_126X_mcRun3_2023_13_1_0_pre2_HBZS0p5_20230330_L1JetEt_PUS_PhiRing_v0_HBE_logGenEtByL1Et_atPU33_HF_GenEtByL1Et_atPU33_woHyperParamOptmz.csv',
        "HBE": {
            "PtAll": {
                "fileName" : '../data/BDTModel_vL1JetEt_PUS_PhiRing_GenJetEt_MLTarget_logGenEtByL1Et_dataFrac1.00_20230403_HBEF_PtAll.pkl',
                "MLTarget": 'logGenEtByL1Et',
                "PUForSFComputation": 33
            },
        },
        "HF": {
            "PtAll": {
                "fileName" : '../data/BDTModel_vL1JetEt_PUS_PhiRing_GenJetEt_MLTarget_GenEtByL1Et_dataFrac1.00_20230403_HBEF_PtAll.pkl',
                "MLTarget": 'GenEtByL1Et',
                "PUForSFComputation": 33
            },
        },        
    },
    
}
sOutDir = "plots_CompareJEC_SFs_v0"

printLevel = PrintLevel = 5
iEtaBins = [i for i in range(1, 42) if i!=29]
sL1JetEt_PUS_ChunkyDonut = 'L1JetEt_PUS_ChunkyDonut'
sL1JetEt_PUS_PhiRing     = 'L1JetEt_PUS_PhiRing'
sOfflineJetEt            = 'PFJetEtCorr'
sGenJetEt                = 'GenJetEt'
sL1JetTowerIEtaAbs       = 'L1JetTowerIEtaAbs'
L1JetPtThrsh             = 10.0 # GeV
L1JetPtMax               = 255.0 # GeV
RefJetPtThrsh            = 10.0 # GeV
snVtx                    = 'nVertexReco'


sSF                    = "SF"


NCompPtBins = 16 # 16 # No. of compressed pT bins
calibSF_L1JetPtRange = [15., 255., 1.] # [<lowest pT>,  <hightest pT>,  <pT bin width>] # pT range for SFs to read from Syed's SF.csv file
LUT_PtRange = [0., 255., 1.] # pT range for SFs for LUT
SF_forZeroPt = 1.0


IEta_Cat_forML = OD()
#IEta_Cat_forML['HB'] = [ 1, 16]
#IEta_Cat_forML['HE12a'] = [17, 26]
#IEta_Cat_forML['HE2b'] = [27, 28]
#IEta_Cat_forML['HF30to32'] = [30, 32]
#IEta_Cat_forML['HF33to36'] = [33, 36]
#IEta_Cat_forML['HF37to41'] = [37, 41]
#IEta_Cat['HBEF'] = [ 1, 41]
IEta_Cat_forML['HBEF'] = [ 1, 41]
IEta_Cat_forML['HBE']  = [ 1, 28]
IEta_Cat_forML['HF']   = [30, 41]

Pt_Cat_forML = OD()
#Pt_Cat_forML['Ptlt60'] = [ 0, 60]
#Pt_Cat_forML['Ptgt60'] = [60, L1JetPtMax]
#Pt_Cat_forML['Ptlt25']   = [ 0, 25]
#Pt_Cat_forML['Pt25to35'] = [25, 35]
#Pt_Cat_forML['Pt35to60'] = [35, 60]
#Pt_Cat_forML['Pt60to90'] = [60, 90]
#Pt_Cat_forML['Ptgt90']   = [90, L1JetPtMax]
Pt_Cat_forML['PtAll'] = [10, L1JetPtMax]



print(f"JEC_SFs_files_dict: {json.dumps(JEC_SFs_files_dict, indent=4) }")

JEC_SFs_files_dict: {
    "2022_ChunkyDonut": {
        "HBEF": {
            "PtAll": {
                "fileName": "/home/siddhesh/Work/CMS/L1_Trigger_Work/L1T_ServiceTask/hcalPUsub/myAna/hcalPUsub_v5_20220311/run_3/l1JetShapesStudies/data/L1T_Jet_SFs_Run3_QCD_Pt15to7000_PFA1p_CMSSW12_6_0_pre1_nVtxAll_20220925_v3_HyperOpt_L1JetEt_PUS_ChunkyDonut.csv"
            }
        }
    },
    "2023_ChunkyDonut": {
        "saveAs": "../data/L1T_Jet_SFs_2023_QCDP_126X_mcRun3_2023_13_1_0_pre2_HBZS0p5_20230330_L1JetEt_PUS_ChunkyDonut_v0_HBE_logGenEtByL1Et_atPU33_HF_GenEtByL1Et_atPU33_woHyperParamOptmz.csv",
        "HBE": {
            "PtAll": {
                "fileName": "../data/BDTModel_vL1JetEt_PUS_ChunkyDonut_GenJetEt_MLTarget_logGenEtByL1Et_dataFrac1.00_20230403_HBEF_PtAll.pkl",
                "MLTarget": "logGenEtByL1Et",
                "PUForSFComputation": 33
            }
        },
        "HF": {
            "PtAll": {
                "fileName": "../data/BDTModel_vL1JetEt_PUS_C

In [2]:
JEC_SFs_dict = {}

for JECVersionName in JEC_SFs_files_dict:
    sL1JetEt                 = sL1JetEt_PUS_ChunkyDonut if 'Donut' in JECVersionName else sL1JetEt_PUS_PhiRing
    
    print(f"{JECVersionName = } ")
    print(f"{sL1JetEt = }")
    
    JEC_SFs_toUse = None
    
    for iEtaCat in JEC_SFs_files_dict[JECVersionName]:
        if iEtaCat not in IEta_Cat_forML: 
            print(f"iEtaCat {iEtaCat} not in IEta_Cat_forML {IEta_Cat_forML}")
            continue
            
        iEtaBinRange = list(range(IEta_Cat_forML[iEtaCat][0], IEta_Cat_forML[iEtaCat][1]+1))
        print(f"{iEtaCat = }, {iEtaBinRange = }")
        
        for PtCat in JEC_SFs_files_dict[JECVersionName][iEtaCat]:
            PtRangeMin   = Pt_Cat_forML[PtCat][0]
            PtRangeMax   = Pt_Cat_forML[PtCat][1]   
            fileName_i           = JEC_SFs_files_dict[JECVersionName][iEtaCat][PtCat]["fileName"]
            MLTarget_i           = JEC_SFs_files_dict[JECVersionName][iEtaCat][PtCat]["MLTarget"] if "MLTarget" in JEC_SFs_files_dict[JECVersionName][iEtaCat][PtCat] else ""
            PUForSFComputation_i = JEC_SFs_files_dict[JECVersionName][iEtaCat][PtCat]["PUForSFComputation"] if "PUForSFComputation" in JEC_SFs_files_dict[JECVersionName][iEtaCat][PtCat] else 0
            
            print(f"{PtCat = }: {PtRangeMin = }, {PtRangeMax = }")
            print(f"{fileName_i = }, {MLTarget_i = }, {PUForSFComputation_i = }")
            data_SFs_i = None
            
            if '.pkl' in fileName_i:
                xgb_i = pickle.load(open(fileName_i, "rb"))                  
                train_vars = xgb_i.get_booster().feature_names
                #print(f"xgb_i: {xgb_i}")                
                print(f"xgb_i.get_booster().feature_names: {xgb_i.get_booster().feature_names}")
                
                # prepareDataframeForSFs -------
                snVtx_toUse = snVtx if snVtx in train_vars else ''
                data_SFs_i = prepareDataframeForSFs(
                    sL1JetTowerIEtaAbs=sL1JetTowerIEtaAbs, iEtaBinRange=iEtaBinRange, 
                    sL1JetEt=sL1JetEt, PtRangeMin=PtRangeMin, PtRangeMax=PtRangeMax, 
                    snVtx=snVtx_toUse, nVtx=PUForSFComputation_i
                )                                
                
                sL1JetEt_forML  = None
                if   MLTarget_i == 'GenEt':
                    sL1JetEt_forML  = sL1JetEt    
                    data_SFs_i[sL1JetEt_forML] = data_SFs_i[sL1JetEt]

                elif MLTarget_i == 'logGenEt':    
                    sL1JetEt_forML  = 'log%s' % (sL1JetEt)
                    data_SFs_i[sL1JetEt_forML] = convert_Et_to_logEt( data_SFs_i[sL1JetEt] )

                elif MLTarget_i == 'GenEtByL1Et':    
                    sL1JetEt_forML  = sL1JetEt
                    data_SFs_i[sL1JetEt_forML] = data_SFs_i[sL1JetEt]

                elif MLTarget_i == 'logGenEtByL1Et':    
                    sL1JetEt_forML  = 'log%s' % (sL1JetEt)
                    data_SFs_i[sL1JetEt_forML] = convert_Et_to_logEt( data_SFs_i[sL1JetEt] )
                
                if sL1JetEt_forML not in train_vars:
                    print(f" {sL1JetEt_forML} not in train_vars {train_vars} \t\t **** ERROR **** ")
                    exit(0)
                    
                # xgb_i.predict() -----
                sL1JetEt_forML_predict = "%s_predict" % (sL1JetEt_forML)
                sL1JetEt_predict       = "%s_predict" % (sL1JetEt)

                data_SFs_i[sL1JetEt_forML_predict] = xgb_i.predict(data_SFs_i[train_vars])
                
                if   MLTarget_i == 'GenEt':
                    data_SFs_i[sL1JetEt_predict] = data_SFs_i[sL1JetEt_forML_predict]

                elif MLTarget_i == 'logGenEt':
                    data_SFs_i[sL1JetEt_predict] = convert_logEt_to_Et( data_SFs_i[sL1JetEt_forML_predict] )

                elif MLTarget_i == 'GenEtByL1Et':
                    data_SFs_i[sL1JetEt_predict] = convert_GenEtByL1Et_to_GenEt( data_SFs_i[sL1JetEt_forML_predict], data_SFs_i[sL1JetEt] )

                elif MLTarget_i == 'logGenEtByL1Et':
                    data_SFs_i[sL1JetEt_predict] = convert_logGenEtByL1Et_to_GenEt( data_SFs_i[sL1JetEt_forML_predict], data_SFs_i[sL1JetEt] )

                
                # SF --------------------
                data_SFs_i[sSF]                    = data_SFs_i[sL1JetEt_predict] / data_SFs_i[sL1JetEt]
                if printLevel >= 11:
                    print("iEtaBins_i: {}".format(iEtaBins_i))
                    print("data_SFs_i: {}".format(data_SFs_i.describe()))
                    
                    
            elif '.csv' in fileName_i:
                # prepareDataframeForSFs -------
                snVtx_toUse = ''
                data_SFs_i = prepareDataframeForSFs(
                    sL1JetTowerIEtaAbs=sL1JetTowerIEtaAbs, iEtaBinRange=iEtaBinRange, 
                    sL1JetEt=sL1JetEt, PtRangeMin=PtRangeMin, PtRangeMax=PtRangeMax, 
                    snVtx=snVtx_toUse, nVtx=PUForSFComputation_i
                )                                
                data_SFs_i[sSF] = 1.0                

                # read JEC SF ---------------------
                with open(fileName_i, mode='r') as fipFileCalibSF_toRead:
                    calibSF_csv_reader = csv.DictReader(fipFileCalibSF_toRead)
                    
                    for calibSF_csv_row in calibSF_csv_reader:
                        iEta_tmp = int(calibSF_csv_row[sL1JetTowerIEtaAbs]) # str
                        l1JetPt_tmp = float(calibSF_csv_row[ sL1JetEt ])
                        SF_tmp = float(calibSF_csv_row[ sSF ]) 
                        
                        data_SFs_i.loc[ (data_SFs_i[sL1JetTowerIEtaAbs] == iEta_tmp) & (abs(data_SFs_i[sL1JetEt] - l1JetPt_tmp) < 1e-3), sSF] = SF_tmp
                        
                #print(f"After {data_SFs_i.describe() = }")
                #print(f"{data_SFs_i.head() = }")
                
            if JEC_SFs_toUse is None:
                JEC_SFs_toUse = data_SFs_i
            else:
                JEC_SFs_toUse = pd.concat([JEC_SFs_toUse, data_SFs_i])    

                
    print(f"JEC_SFs_toUse: {JEC_SFs_toUse.describe()}"  )
    JEC_SFs_dict[JECVersionName] = JEC_SFs_toUse
    
    if "saveAs" in JEC_SFs_files_dict[JECVersionName]:
        sOpFileName_SFs = JEC_SFs_files_dict[JECVersionName]["saveAs"]
        JEC_SFs_toUse.to_csv(sOpFileName_SFs, index=False)
        print("Wrote {} \n".format(sOpFileName_SFs)) 
                
            
                
    
    



JECVersionName = '2022_ChunkyDonut' 
sL1JetEt = 'L1JetEt_PUS_ChunkyDonut'
iEtaCat = 'HBEF', iEtaBinRange = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
PtCat = 'PtAll': PtRangeMin = 10, PtRangeMax = 255.0
fileName_i = '/home/siddhesh/Work/CMS/L1_Trigger_Work/L1T_ServiceTask/hcalPUsub/myAna/hcalPUsub_v5_20220311/run_3/l1JetShapesStudies/data/L1T_Jet_SFs_Run3_QCD_Pt15to7000_PFA1p_CMSSW12_6_0_pre1_nVtxAll_20220925_v3_HyperOpt_L1JetEt_PUS_ChunkyDonut.csv', MLTarget_i = '', PUForSFComputation_i = 0
JEC_SFs_toUse:        L1JetTowerIEtaAbs  L1JetEt_PUS_ChunkyDonut            SF
count       10086.000000             10086.000000  10086.000000
mean           21.000000               132.500000      1.241402
std            11.832746                71.017017      0.340388
min             1.000000                10.000000      0.585001
25%            11.000000                71.000000      

In [3]:
marker_color_list = ['r', 'b', 'darkviolet', 'c', 'orange', 'green']
marker_style_list = ["o", "o", "o",          "o", "o",      "o", "o", "X", '>', '^', 'v', "s", "+", 'x', '*']
marker_size_list  = [3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2]

sOutDir_toUse = sOutDir
os.makedirs(sOutDir_toUse, exist_ok=True)

for iEtaBin in iEtaBins:
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(5,4), layout='constrained')
    yMin_ =  9999
    yMax_ = -9999
    i_ = 0
    
    for JECVersionName in JEC_SFs_files_dict:
        sL1JetEt                 = sL1JetEt_PUS_ChunkyDonut if 'Donut' in JECVersionName else sL1JetEt_PUS_PhiRing
        #JEC_SFs_dict[JECVersionName]
        
        JEC_SFs_toUse_1 = JEC_SFs_dict[JECVersionName]
        JEC_SFs_toUse = JEC_SFs_toUse_1[(
            (JEC_SFs_toUse_1[sL1JetTowerIEtaAbs] == iEtaBin)
        )]
        
        print(f"{JEC_SFs_toUse[sL1JetEt] = }")
        print(f"{JEC_SFs_toUse[sSF] = }")
        
        axs.plot(JEC_SFs_toUse[sL1JetEt], JEC_SFs_toUse[sSF], label=JECVersionName, color='None', marker=marker_style_list[i_], markersize=marker_size_list[i_], markerfacecolor=marker_color_list[i_], markeredgecolor='None')        
        i_ += 1
        
        yMin1_ = np.min(JEC_SFs_toUse[sSF])
        yMax1_ = np.max(JEC_SFs_toUse[sSF])        
        if yMin1_ < yMin_: yMin_ = yMin1_
        if yMax1_ > yMax_: yMax_ = yMax1_

        
    axs.set_xlabel('%s [GeV]' % ('L1T pT'))
    axs.set_ylabel(r'SF = GEN pT/L1T pT')
    axs.set_title('iEta %d ' % (iEtaBin))
    if yMin_ < 0.0 or yMax_ > 4.0:
        axs.set_ylim(0., 4)
    else:
        axs.set_ylim(yMin_, yMax_)
    axs.set_xlim(15, 255)
    axs.legend(bbox_to_anchor=(0.1, 1.05), loc='upper left', borderaxespad=0.9, ncol=2)
    axs.margins(y=0.3)
    axs.grid()
    fig.savefig('%s/JECSFs_vs_pT_iEta_%d.png' % (sOutDir_toUse, iEtaBin))   
    plt.close(fig)    
        
        

    

JEC_SFs_toUse[sL1JetEt] = 0       10.0
1       11.0
2       12.0
3       13.0
4       14.0
       ...  
241    251.0
242    252.0
243    253.0
244    254.0
245    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 0      1.000000
1      1.720556
2      1.722151
3      1.741592
4      1.742634
         ...   
241    1.228152
242    1.223279
243    1.218443
244    1.227526
245    1.222712
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 0       10.0
1       11.0
2       12.0
3       13.0
4       14.0
       ...  
241    251.0
242    252.0
243    253.0
244    254.0
245    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 0      2.223923
1      2.174299
2      2.099507
3      2.028252
4      1.996331
         ...   
241    1.226833
242    1.226833
243    1.226833
244    1.226833
245    1.224734
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 0       10.0
1       11.0
2       12.0
3       1

JEC_SFs_toUse[sL1JetEt] = 1230     10.0
1231     11.0
1232     12.0
1233     13.0
1234     14.0
        ...  
1471    251.0
1472    252.0
1473    253.0
1474    254.0
1475    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 1230    1.000000
1231    1.817612
1232    1.812404
1233    1.833011
1234    1.831108
          ...   
1471    1.245294
1472    1.240353
1473    1.235450
1474    1.244750
1475    1.239869
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 1230     10.0
1231     11.0
1232     12.0
1233     13.0
1234     14.0
        ...  
1471    251.0
1472    252.0
1473    253.0
1474    254.0
1475    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 1230    2.344902
1231    2.255843
1232    2.187091
1233    2.124040
1234    2.086577
          ...   
1471    1.248835
1472    1.248835
1473    1.248835
1474    1.248835
1475    1.249283
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 1230

JEC_SFs_toUse[sL1JetEt] = 2460     10.0
2461     11.0
2462     12.0
2463     13.0
2464     14.0
        ...  
2701    251.0
2702    252.0
2703    253.0
2704    254.0
2705    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 2460    1.000000
2461    1.803809
2462    1.831186
2463    1.854042
2464    1.874108
          ...   
2701    1.259998
2702    1.254998
2703    1.250038
2704    1.259616
2705    1.254676
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 2460     10.0
2461     11.0
2462     12.0
2463     13.0
2464     14.0
        ...  
2701    251.0
2702    252.0
2703    253.0
2704    254.0
2705    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 2460    2.410802
2461    2.330287
2462    2.270755
2463    2.218675
2464    2.183331
          ...   
2701    1.259025
2702    1.259025
2703    1.259025
2704    1.259025
2705    1.259477
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 2460

JEC_SFs_toUse[sL1JetEt] = 3690     10.0
3691     11.0
3692     12.0
3693     13.0
3694     14.0
        ...  
3931    251.0
3932    252.0
3933    253.0
3934    254.0
3935    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 3690    1.000000
3691    2.231669
3692    2.266276
3693    2.285706
3694    2.267364
          ...   
3931    1.417849
3932    1.412222
3933    1.406641
3934    1.416210
3935    1.410657
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 3690     10.0
3691     11.0
3692     12.0
3693     13.0
3694     14.0
        ...  
3931    251.0
3932    252.0
3933    253.0
3934    254.0
3935    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 3690    2.743473
3691    2.688492
3692    2.610146
3693    2.575832
3694    2.536059
          ...   
3931    1.409033
3932    1.409033
3933    1.409033
3934    1.409033
3935    1.410309
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 3690

JEC_SFs_toUse[sL1JetEt] = 5166     10.0
5167     11.0
5168     12.0
5169     13.0
5170     14.0
        ...  
5407    251.0
5408    252.0
5409    253.0
5410    254.0
5411    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 5166    1.000000
5167    2.478240
5168    2.432093
5169    2.386685
5170    2.319342
          ...   
5407    1.236922
5408    1.243306
5409    1.238470
5410    1.233807
5411    1.239054
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 5166     10.0
5167     11.0
5168     12.0
5169     13.0
5170     14.0
        ...  
5407    251.0
5408    252.0
5409    253.0
5410    254.0
5411    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 5166    2.706685
5167    2.539159
5168    2.459091
5169    2.364980
5170    2.296861
          ...   
5407    1.173346
5408    1.173346
5409    1.173346
5410    1.173346
5411    1.173767
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 5166

JEC_SFs_toUse[sL1JetEt] = 6396     10.0
6397     11.0
6398     12.0
6399     13.0
6400     14.0
        ...  
6637    251.0
6638    252.0
6639    253.0
6640    254.0
6641    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 6396    1.000000
6397    2.419374
6398    2.247742
6399    2.047332
6400    2.002206
          ...   
6637    1.091903
6638    1.099904
6639    1.095713
6640    1.091587
6641    1.091649
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 6396     10.0
6397     11.0
6398     12.0
6399     13.0
6400     14.0
        ...  
6637    251.0
6638    252.0
6639    253.0
6640    254.0
6641    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 6396    1.798704
6397    1.641378
6398    1.512642
6399    1.394081
6400    1.292695
          ...   
6637    0.629611
6638    0.629611
6639    0.629611
6640    0.629611
6641    0.629837
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 6396

JEC_SFs_toUse[sL1JetEt] = 7872     10.0
7873     11.0
7874     12.0
7875     13.0
7876     14.0
        ...  
8113    251.0
8114    252.0
8115    253.0
8116    254.0
8117    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 7872    1.000000
7873    1.006387
7874    0.976643
7875    0.912858
7876    0.859368
          ...   
8113    0.857148
8114    0.853747
8115    0.850372
8116    0.849165
8117    0.845835
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 738     10.0
739     11.0
740     12.0
741     13.0
742     14.0
       ...  
979    251.0
980    252.0
981    253.0
982    254.0
983    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 738    1.628111
739    1.467833
740    1.329962
741    1.248427
742    1.172351
         ...   
979    0.781217
980    0.781217
981    0.781217
982    0.781217
983    0.764852
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 7872     10.0
7873     11.

JEC_SFs_toUse[sL1JetEt] = 9102     10.0
9103     11.0
9104     12.0
9105     13.0
9106     14.0
        ...  
9343    251.0
9344    252.0
9345    253.0
9346    254.0
9347    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 9102    1.000000
9103    1.141795
9104    1.094763
9105    1.035708
9106    0.984664
          ...   
9343    0.785658
9344    0.782540
9345    0.779447
9346    0.776378
9347    0.773334
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 1968     10.0
1969     11.0
1970     12.0
1971     13.0
1972     14.0
        ...  
2209    251.0
2210    252.0
2211    253.0
2212    254.0
2213    255.0
Name: L1JetEt_PUS_ChunkyDonut, Length: 246, dtype: float64
JEC_SFs_toUse[sSF] = 1968    1.389858
1969    1.300999
1970    1.235257
1971    1.168003
1972    1.107754
          ...   
2209    0.763323
2210    0.763323
2211    0.763323
2212    0.763323
2213    0.749401
Name: SF, Length: 246, dtype: float64
JEC_SFs_toUse[sL1JetEt] = 9102